In [2]:
'''imports'''

import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [8]:
'''entrada da primeira página de resultado de busca e geração de lista de matérias'''

url_input = str(input('Entre a URL da primeira página da busca: '))
qtd_paginas = int(np.ceil(int(input('Quantos resultados existem nessa busca? '))/25))

materias=[] # lista para armazenar as URLs das matérias
re_compile = 'http://www1.folha.uol.com.br/fsp/'
# geração das urls de página de busca:
url_busca = [str(url_input)+'&sr='+str((i*25)+1) for i in range(qtd_paginas)]
    
# laço para varrer as páginas de busca e empilhar as urls na lista materias:
for url in url_busca:
    conteudo_bs = BeautifulSoup(requests.get(url).content, 'html.parser')

    # busca a url das matérias dentro da página de busca (apenas contendo www1..../fsp/):
    for materia in conteudo_bs.findAll('a', attrs={'href': re.compile(re_compile)}):
        materias.append(materia.get('href'))
        
print('Listagem de matérias realizada! Matérias adicionadas à lista: '+ str(len(materias)))

Entre a URL da primeira página da busca: https://search.folha.uol.com.br/search?q=copa+do+mundo+futebol&periodo=personalizado&sd=12%2F06%2F1994&ed=12%2F07%2F1994&site=todos
Quantos resultados existem nessa busca? 320
Listagem de matérias realizada! Matérias adicionadas à lista: 316


In [9]:
'''abre matérias da lista, busca, limpa e salva o título em uma lista e o texto em outra'''

titulos=[] # lista dos títulos das matérias
textos=[] # lista contendo o texto das matérias

# laço abrindo matéria por matéria da lista materias:
for mat in materias:
    
    materia_bs = str(BeautifulSoup(requests.get(mat).content)).replace('<br/>', ' ')

    # busca o título e armazena na lista titulos:
    titulos.append(BeautifulSoup(materia_bs).find('title').text[19:-11])

    # condição para armazenar matérias específicas; algumas não possuem Texto Anterior: no corpo
    if 'Texto Anterior:' in materia_bs:
        t1 = BeautifulSoup(materia_bs).find(text=re.compile('Texto Anterior:'))
        textos.append(BeautifulSoup(t1[:t1.index('Texto Anterior')],'html.parser'))
    elif 'Próximo Texto:' in materia_bs:
        t1 = BeautifulSoup(materia_bs).find(text=re.compile('Próximo Texto:'))
        textos.append(BeautifulSoup(t1[:t1.index('Próximo ')],'html.parser'))
    else:
        continue

In [10]:
'''cria os arquivos e armazena neles o título seguido do corpo de texto'''

arquivos=[] # inicia a lista para criação de arquivos e armazenamento dos textos
dir=input(str('Qual o diretório onde serão criados os arquivos? Colocar o endereço com a "\\" no final: '))
size=len(materias)

for i in range(size):
    arquivo_nome='1994_'+str(i+1)+'.txt'
    arquivo=open(dir+arquivo_nome,'w+', encoding='utf-8')
    empilhado=open(dir+'1994_empilhado.txt','a', encoding='utf-8')
    arquivo.write(str(titulos[i])+str(textos[i]))
    empilhado.write(str(titulos[i])+str(textos[i]))
    arquivo.close()
    empilhado.close()
    arquivos.append(arquivo_nome)

Qual o diretório onde serão criados os arquivos? Colocar o endereço com a "\" no final: C:\Users\ruan_\Desktop\Projeto\textos\1994\
